In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import src.kgbench as kg
import fire, sys
import math

from kgbench import load, tic, toc, d


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors


# #
#from torch_geometric.utils import to_networkx
# import networkx as nx

from src.rgcn_explainer_utils import *
from IPython.core.interactiveshell import InteractiveShell
from rgcn_model import RGCN



In [3]:
import os

# Get the current directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Check if the current directory is already the parent directory
if current_dir != '/Users/macoftraopia/Documents/GitHub/RGCN-Explainer':
    # Set the parent directory as the current directory
    os.chdir(parent_dir)
!pwd

/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


In [4]:
import kgbench as kg
from src.rgcn_explainer_utils import *

data = kg.load('aifb', torch=True, final=True)
data = prunee(data, 2)
get_relations(data)
relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]

loaded data aifb (0.3363s).


/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/kgbench/load.py:123: UserWarning: The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, the correct evaluation is to combine train and validation for the final evaluation run.Set include_val to True when loading the data.
  warnings.warn('The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, '


In [11]:
name = 'aifb'

data = kg.load(name, torch=True) 
print(data.triples.shape[0])
node_idx = 5678

# else:
#     data = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/data/IMDB/finals/{name}.pt')

data = prunee(data, 2)
print(data.triples.shape[0])
data.triples = torch.Tensor(data.triples).to(int)#data.triples.clone().detach()
data.withheld = torch.Tensor(data.withheld).to(int)#data.withheld.clone().detach()
data.training = torch.Tensor(data.training).to(int)#data.training.clone().detach()
#
get_relations(data)
d_classes(data)
dict_classes = {key.item(): data.withheld[:, 0][data.withheld[:, 1] == key].tolist() for key in torch.unique(data.withheld[:, 1])}
if name != 'aifb':
    node_idx = dict_classes[0][0]


from src.rgcn_explainer_utils import *
# v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_ver{node_idx}')
# h = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_hor{node_idx}')
model = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/model_{name}_prune_True')

# h = select_entity(h, 5230)
# v = select_entity(v, 5230)

v = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_hor{node_idx}')

# v = torch.sparse_coo_tensor(v.coalesce().indices(), torch.sigmoid(v.coalesce().values()), v.size(), requires_grad=True)
# h = torch.sparse_coo_tensor(h.coalesce().indices(), torch.sigmoid(h.coalesce().values()), h.size(), requires_grad=True)
out = model.forward2(h,v)

res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)
# print(v.coalesce().values()[v.coalesce().values()>0.5])
# print(h.coalesce().values())

v_bin,h_bin = convert_binary(v, 0.5), convert_binary(h,0.5)
print('v binary:',v_bin, torch.count_nonzero(v_bin.coalesce().values()))
res = nn.Softmax(dim=0)(model.forward2(h_bin,v_bin)[node_idx, :])
print('ypred explain binary', res)

if node_idx in data.withheld[:,0]:
    print('ypred true', data.withheld[data.withheld[:,0]==node_idx,1])
    

model.eval()
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[node_idx, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)

v,h = sub(v, 0.5), sub(h,0.5)
m = match_to_triples(v,h,data, node_idx)
Counter(m[:,1].tolist())

loaded data aifb (0.2276s).
29043
26666
ypred explain tensor([0.4507, 0.1969, 0.2936, 0.0587], grad_fn=<SoftmaxBackward0>)
v binary: tensor(indices=tensor([[ 23451,  23451,  23451,  ..., 329925, 329926, 329927],
                       [  5678,   5743,   5746,  ...,   5230,   5230,   5230]]),
       values=tensor([1., 1., 1.,  ..., 0., 0., 0.]),
       size=(753935, 8285), nnz=1311, layout=torch.sparse_coo) tensor(119)
ypred explain binary tensor([0.3810, 0.2180, 0.3542, 0.0468], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
ypred full tensor([9.8647e-01, 5.4700e-04, 1.2982e-02, 1.1929e-06],
       grad_fn=<SoftmaxBackward0>)


Counter({2: 27, 10: 1, 15: 1, 18: 35, 21: 21, 27: 1, 30: 30, 36: 3, 0: 101})

In [12]:
node_idx = 5678
init = 'normal'
v = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_hor{node_idx}')
#v_bin,h_bin = convert_binary(v, 0.5), convert_binary(h,0.5)

In [14]:
def random_explanation_baseline(sparse_tensor):
    ''' Create a random explanation baseline for a given sparse tensor'''
    # Retrieve the indices of non-zero elements
    # explanation_lenght = len(sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values()>config['threshold'] ])
    explanation_lenght = len(sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values()> 0.5 ])
    indices = sparse_tensor._indices()

    # Get the total number of non-zero elements
    num_nonzero = indices.size(1)

    # Specify the number of random indices you want to select
    n = explanation_lenght

    # Generate 'n' random indices within the range of non-zero indices
    random_indices = torch.randperm(num_nonzero)[:n]

    # Create a new sparse tensor with the same shape as the original tensor but with all values set to 0
    new_sparse_tensor = torch.sparse.FloatTensor(indices, torch.zeros(num_nonzero), size=sparse_tensor.size())

    # Assign 1 to the randomly selected indices in the new sparse tensor
    new_sparse_tensor._values()[random_indices] = 1

    # Print the new sparse tensor
    return new_sparse_tensor

h_random, v_random = random_explanation_baseline(h), random_explanation_baseline(v)
counter = important_relation(h_random, v_random, data,node_idx, 0.5)
print('Important relations', counter)
res_random = nn.Softmax(dim=0)(model.forward2(h_random, v_random)[node_idx, :])
print(res_random)

Important relations {'author': 7, 'homepage': 1, 'isWorkedOnBy': 5, 'member': 2, 'publication': 4, 'type': 100}
tensor([0.2483, 0.2657, 0.2633, 0.2227], grad_fn=<SoftmaxBackward0>)


In [5]:
def frequency_relations(data):
    freq = Counter(data.triples[:,1].tolist())
    sorted_freq = {data.i2r[k]: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}
    return sorted_freq
f = frequency_relations(data)
print(f)

{'http://swrc.ontoware.org/ontology#publication': 4139, 'http://swrc.ontoware.org/ontology#author': 3962, 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': 3785, 'http://swrc.ontoware.org/ontology#isAbout': 2475, 'http://swrc.ontoware.org/ontology#publishes': 1217, 'http://swrc.ontoware.org/ontology#year': 1210, 'http://swrc.ontoware.org/ontology#title': 1136, 'http://swrc.ontoware.org/ontology#projectInfo': 952, 'http://swrc.ontoware.org/ontology#hasProject': 952, 'http://swrc.ontoware.org/ontology#month': 746, 'http://swrc.ontoware.org/ontology#booktitle': 721, 'http://swrc.ontoware.org/ontology#isWorkedOnBy': 567, 'http://swrc.ontoware.org/ontology#pages': 514, 'http://swrc.ontoware.org/ontology#abstract': 492, 'http://swrc.ontoware.org/ontology#dealtWithIn': 355, 'http://swrc.ontoware.org/ontology#member': 338, 'http://swrc.ontoware.org/ontology#name': 335, 'http://swrc.ontoware.org/ontology#volume': 298, 'http://swrc.ontoware.org/ontology#series': 282, 'http://swrc.ontoware.org/o

In [23]:
import kgbench as kg
from src.rgcn_explainer_utils import *

data = kg.load('aifb', torch=True, final=True)
data = prunee(data, 2)
get_relations(data)
relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]
d = d_classes(data)
node_idx = d[list(d.keys())[0]][0] 

/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/kgbench/load.py:123: UserWarning: The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, the correct evaluation is to combine train and validation for the final evaluation run.Set include_val to True when loading the data.
  warnings.warn('The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, '


loaded data aifb (0.777s).


In [24]:
#but I want to get the most frequent relations for a given node (2 hops)

def most_frequent_relations(data, node_idx, n_hops):
    ''' Most frequent relations for a given node (2 hops)'''
    hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
    edge_index_h, edge_index_v = hor_graph.coalesce().indices(), ver_graph.coalesce().indices()

    sub_edges_h, neighbors_h, sub_edges_tensor_h  = find_n_hop_neighbors(edge_index_h, n=n_hops, node=node_idx)
    sub_edges_v, neighbors_v, sub_edges_tensor_v  = find_n_hop_neighbors(edge_index_v, n=n_hops, node=node_idx)
    sub_triples = match_to_triples(sub_edges_tensor_v, sub_edges_tensor_h,data, sparse=False)
    sub_h, sub_v = hor_ver_graph(sub_triples, data.num_entities, data.num_relations)
    m = match_to_triples(sub_v, sub_h,data, node_idx)
    freq = Counter(m[:,1].tolist())
    sorted_freq = {data.i2r[k]: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True) if k!=0}

    most_freq_rel = list(sorted_freq.keys())[0]
    id_most_freq_rel = data.r2i[most_freq_rel]
    return most_freq_rel
most_freq_rel = most_frequent_relations(data, node_idx, 2)

pv is empty


In [25]:
print('most frequent relation', most_freq_rel)

most frequent relation http://www.w3.org/1999/02/22-rdf-syntax-ns#type


In [55]:
def degree_distribution(data):
    edge_index = edge_index_oneadj(data.triples)
    degree = []
    for i in range(data.num_entities):
        _,n,_ = find_n_hop_neighbors(edge_index, 0, i)
        degree.append(len(n))
    degree_mean = np.mean(degree)
    return degree_mean

degree_distribution(data)

KeyError: 23643

In [30]:
counter

[tensor(4945),
 tensor(5230),
 tensor(4946),
 tensor(4951),
 tensor(4973),
 tensor(4974),
 tensor(4987),
 tensor(4988),
 tensor(4998),
 tensor(4999),
 tensor(5005),
 tensor(5010),
 tensor(5014),
 tensor(5016),
 tensor(5231),
 tensor(5022),
 tensor(5034),
 tensor(5238),
 tensor(5035),
 tensor(5200),
 tensor(5036),
 tensor(5235),
 tensor(5042),
 tensor(5053),
 tensor(5063),
 tensor(5078),
 tensor(5081),
 tensor(5085),
 tensor(8176),
 tensor(5211),
 tensor(5202),
 tensor(5203),
 tensor(5215),
 tensor(5206),
 tensor(5216),
 tensor(5217),
 tensor(5246),
 tensor(5219),
 tensor(5220),
 tensor(5223),
 tensor(5228),
 tensor(5027),
 tensor(5028),
 tensor(5029),
 tensor(5030),
 tensor(5031),
 tensor(5032),
 tensor(5033),
 tensor(5233),
 tensor(5240),
 tensor(5242),
 tensor(5247),
 tensor(5349),
 tensor(5950),
 tensor(5964),
 tensor(5970),
 tensor(5977),
 tensor(5679),
 tensor(5732),
 tensor(5748),
 tensor(5791),
 tensor(5855),
 tensor(5886),
 tensor(5900),
 tensor(8281),
 tensor(5243),
 tensor(53

In [22]:
counter = []
for i in data.triples:
    if i[0] not in counter:
        counter.append(i[0])
    if i[2] not in counter:
        counter.append(i[2])

len(set(counter))

6575

In [ ]:
v.coalesce().values()

In [ ]:


def find_threshold(sparse_tensor):
    ''' Find the threshold value for the sparse tensor'''
    # sparse_tensor = torch.sparse_coo_tensor(
    #     sparse_tensor.coalesce().indices()%data.num_entities, sparse_tensor.coalesce().values(), size=sparse_tensor.size()
    # )
    numbers = sparse_tensor.coalesce().values()
    sorted_numbers = sorted(numbers, reverse=True)
    count = 0
    threshold = None
    
    for num in sorted_numbers:
        if count == 10:
            break
        threshold = num
        count += 1
    
    return threshold

def convert_back(sparse_tensor, data):
    sparse_tensor = torch.sparse_coo_tensor(
        sparse_tensor.coalesce().indices()%data.num_entities, sparse_tensor.coalesce().values(), size=sparse_tensor.size()
    )
    return sparse_tensor

t =     find_threshold(v)
v, h = convert_back(v, data), convert_back(h, data)
print(sub(v,t),t)
v, h =convert_binary(v,t), convert_binary(h,t)
print(v.coalesce().values().count_nonzero())
v_sub = sub(v,t)
if node_idx in v_sub.coalesce().indices():
    print('node_idx in v')
else:
    print('node_idx not in v')

#print(t, sub(v,t))

out = model.forward2(h,v)

res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)

In [ ]:
v = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_hor{node_idx}')
def find_threshold(sparse_tensor, num_exp):
    ''' Find the threshold value for the sparse tensor'''
    # sparse_tensor = torch.sparse_coo_tensor(
    #     sparse_tensor.coalesce().indices()%data.num_entities, sparse_tensor.coalesce().values(), size=sparse_tensor.size()
    # )
    numbers = sparse_tensor.coalesce().values()
    sorted_numbers = sorted(numbers, reverse=True)
    count = 0
    threshold = None
    
    for num in sorted_numbers:
        if count == num_exp:
            break
        threshold = num
        count += 1
    
    return threshold


def threshold_mask(h,v ,data, num_exp):
    ''' Apply a threshold mask to the adjacency matrix'''
    t =     find_threshold(v, num_exp)
    #v, h = convert_back(v, data), convert_back(h, data)
    v, h =convert_binary(v,t), convert_binary(h,t)
    return h,v

h,v = threshold_mask(h,v, data, 10)
out = model.forward2(h,v)

res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)    

v.coalesce().values().count_nonzero()
masked_ver_sub, masked_hor_sub = sub(v, 0.5), sub(h,0.5)
print(masked_hor_sub.coalesce().values().count_nonzero())
m = match_to_triples(masked_ver_sub, masked_hor_sub, data, node_idx)
m

In [ ]:
v = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/old_all/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_1_type_1_killtype_True_init_normal_break_wrong_pred_exp_/masked_adj/masked_hor{node_idx}')


masked_ver_sub, masked_hor_sub = sub(v, 0.5), sub(h,0.5)
m = match_to_triples(masked_ver_sub, masked_hor_sub, data, node_idx)
counter = dict(Counter(m[:,1].tolist()))
counter = {data.i2rel[k][0]:v for k,v in counter.items() if k!=0}
print('Important relations', counter)

def important_relation(v,h,data):
    masked_ver_sub, masked_hor_sub = sub(v, 0.5), sub(h,0.5)
    m = match_to_triples(masked_ver_sub, masked_hor_sub, data, node_idx)
    counter = dict(Counter(m[:,1].tolist()))
    counter = {data.i2rel[k][0]:v for k,v in counter.items() if k!=0}
    print('Important relations', counter)
    return counter

v,h = threshold_mask(h,v, data, 10)
important_relation(v,h,data)


In [ ]:
import torch

# def find_threshold_sparse_tensor(sparse_tensor, index, n):
#     # Get the values and indices of the sparse tensor
#     values = sparse_tensor.coalesce().values()
#     indices = sparse_tensor.coalesce().indices()

#     # Filter the values and indices based on the specified index
#     selected_values = values[indices[1] == index]

#     # Sort the selected values in descending order
#     sorted_values, _ = torch.sort(selected_values, descending=True)

#     # Find the threshold value
#     if n < sorted_values.size(0):
#         threshold = sorted_values[n - 1]
#     else:
#         threshold = sorted_values[-1]

#     return threshold
def find_threshold_sparse_tensor(sparse_tensor, index, n):
    # Get the values and indices of the sparse tensor
    values = sparse_tensor.coalesce().values()
    indices = sparse_tensor.coalesce().indices()

    # Filter the values and indices based on the specified index
    selected_values = values[indices[0] == index]

    # Sort the selected values in descending order
    sorted_values, _ = torch.sort(selected_values, descending=True)

    # Find the threshold value
    if n <= sorted_values.size(0):
        threshold = sorted_values[n - 1]
    else:
        threshold = sorted_values[-1]

    return threshold

# Example sparse tensor
sparse_tensor = h

# Specify the index and maximum number of values
index = 5757
n = 10
# Find the threshold
t = find_threshold_sparse_tensor(sparse_tensor, index, n)

v, h =convert_binary(v,t), convert_binary(h,t)
sub(h,t)


In [ ]:
import torch

def is_graph_connected(adjacency_matrix):
    n = adjacency_matrix.shape[0]
    visited = torch.zeros(n, dtype=torch.bool)
    stack = [0]  # Start traversal from node 0

    while stack:
        node = stack.pop()
        visited[node] = True

        neighbor_indices = adjacency_matrix.coalesce().indices()
        neighbor_values = adjacency_matrix.coalesce().values()

        node_neighbors = neighbor_indices[0, neighbor_indices[1] == node]
        for neighbor in node_neighbors:
            if not visited[neighbor]:
                stack.append(neighbor)
                print(stack)

    return visited.all()


def remove_disconnected_edges(adjacency_matrix):
    if not is_graph_connected(adjacency_matrix):
        connected_indices = []
        connected_values = []

        for node in range(adjacency_matrix.size(0)):
            row = adjacency_matrix[node]
            node_neighbors = row.coalesce().indices()
            node_values = row.coalesce().values()

            connected_indices.extend([(node, neighbor) for neighbor in node_neighbors])
            connected_values.extend(node_values)

        connected_indices = torch.tensor(connected_indices).t()
        connected_values = torch.tensor(connected_values)
        connected_adjacency_matrix = torch.sparse_coo_tensor(
            connected_indices, connected_values, size=adjacency_matrix.size()
        )

        return connected_adjacency_matrix

    return adjacency_matrix



is_graph_connected(v)


In [ ]:
import torch
node_idx = 5757
v = torch.load(f'chk/aifb_chk/exp/init_normal_hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_10_type_1_killtype_True_break_no/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/exp/init_normal_hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_adaptive_ent_10_type_1_killtype_True_break_no/masked_adj/masked_hor{node_idx}')

def select_connected_subgraph(adjacency_matrix, given_node,data):
    adjacency_matrix = torch.sparse_coo_tensor(
        adjacency_matrix.coalesce().indices()%data.num_entities, adjacency_matrix.coalesce().values(), size=adjacency_matrix.size()
    )
    sub_adj = sub(adjacency_matrix, 0.5)
    print(adjacency_matrix)
    num_nodes = sub_adj.size(0)
    visited = torch.zeros(num_nodes, dtype=torch.bool)
    connected_nodes = set()
    stack = []

    # Starting with the given node
    stack.append(given_node)
    connected_nodes.add(given_node)
    visited[given_node] = True

    while len(stack) > 0:
        node = stack.pop()
        neighbors = sub_adj[node].coalesce().indices()
        for i in range(neighbors.size(1)):
            neighbor = neighbors[:, i]
            if not visited[neighbor[0]]:
                stack.append(neighbor[0])
                connected_nodes.add(neighbor[0])
                visited[neighbor[0]] = True

    # Select the indices of the connected nodes
    connected_indices = []
    for node in connected_nodes:
        connected_indices.append([node, node])

    # Create the connected adjacency matrix
    connected_indices = torch.tensor(connected_indices, dtype=torch.long).t()
    #connected_values = adjacency_matrix._values()[connected_indices[0]]
    connected_values = torch.ones(connected_indices.size(1))

    #torch.ones(connected_indices.size(1))
    disconnected_indices = get_non_selected_indices(adjacency_matrix, connected_indices)
    disconnected_values = torch.zeros(disconnected_indices.size(1))
    connected_indices = torch.cat([connected_indices, disconnected_indices], dim=1)
    connected_values = torch.cat([connected_values, disconnected_values])
    connected_adjacency_matrix = torch.sparse_coo_tensor(
        connected_indices, connected_values, size=adjacency_matrix.size()
    )

    return connected_adjacency_matrix

# Example usage

# v, h = sub(v, 0.5), sub(h,0.5)
v = select_connected_subgraph(v, node_idx,data)
h = select_connected_subgraph(h, node_idx,data)
print(v.coalesce().values().count_nonzero(), h.coalesce().values().count_nonzero())
out = model.forward2(h,v)

res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)

In [ ]:
import torch

# Example sparse tensor
indices = torch.tensor([[0, 1, 1], [1, 0, 1]])
values = torch.tensor([2, 4, 6])
sparse_tensor = torch.sparse_coo_tensor(indices, values, size=(2, 2))

# Example selected indices
selected_indices = torch.tensor([[0, 1], [1, 1]])

# Convert selected indices to a boolean mask
mask = torch.sparse_coo_tensor(selected_indices, torch.ones(selected_indices.size(1)), size=(2, 2)).to_dense().bool()

# Use torch.where to select values based on the mask
selected_values = torch.where(mask, sparse_tensor.coalesce().values(), torch.zeros(mask.sum()))

print(selected_values)


In [ ]:
import torch

def get_non_selected_indices(sparse_tensor, selected_indices):
    original_indices = sparse_tensor.coalesce().indices()
    selected_set = set(map(tuple, selected_indices.t().tolist()))

    non_selected_indices = []
    for index in original_indices.t().tolist():
        if tuple(index) not in selected_set:
            non_selected_indices.append(index)

    return torch.tensor(non_selected_indices).t()

# Example usage
sparse_tensor = torch.sparse_coo_tensor(torch.tensor([[0, 0, 1, 1], [0, 1, 0, 2]]), torch.tensor([2, 3, 4, 5]), size=(2, 3))
selected_indices = torch.tensor([[0, 1], [1, 2]])  # Example selected indices

non_selected_indices = get_non_selected_indices(sparse_tensor, selected_indices)
print(non_selected_indices)


In [ ]:
sv,ov = v_bin.coalesce().indices()%data.num_entities
sh,oh = h_bin.coalesce().indices()%data.num_entities

v = torch.sparse_coo_tensor(torch.stack([sv,ov]), v_bin.coalesce().values(), v_bin.size())
h = torch.sparse_coo_tensor(torch.stack([sh,oh]), h_bin.coalesce().values(), h_bin.size())

for i in v.coalesce().indices():
    if 

In [ ]:
name = 'IMDb_us'
data = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/data/IMDB/finals/{name}.pt')
data

In [ ]:
v = torch.load(f'chk/aifb_chk/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_1_ent_1_type_1_killtype_False_init_normal_break_wrong_pred_exp_/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/hops_2_lr_0.1_adaptive_False_size_5e-05_sizestd_1_ent_1_type_1_killtype_False_init_normal_break_wrong_pred_exp_/masked_adj/masked_hor{node_idx}')
v, h = select_on_relation_sparse(v,data, 39), select_on_relation_sparse(h,data, 39)
#print(v_.coalesce().values().count_nonzero())
v_bin,h_bin = convert_binary(v, 0.5), convert_binary(h,0.5)

print(v_bin.coalesce().values().count_nonzero())
out = model.forward2(h_bin,v_bin)

res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)

In [ ]:
#inverse predictions 
v_inv, h_inv = inverse_tensor(v), inverse_tensor(h)
print(v_inv.coalesce().values().count_nonzero())

out = model.forward2(h_inv,v_inv)

res1_m = nn.Softmax(dim=0)(out[node_idx])
res - res1_m

In [ ]:
def inverse_tensor(sparse_tensor):
    """ Convert 0 to 1 and viceversa in sparse tensor
    The aim is computing the Fidelity- score"""
    sparse_tensor = convert_binary(sparse_tensor, 0.5)
    sparse_tensor = torch.sparse_coo_tensor(indices=sparse_tensor._indices(), 
                                        values=1 - sparse_tensor._values(), 
                                        size=sparse_tensor.size())
    return sparse_tensor


In [ ]:
name = 'IMDb_us'
data = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/data/IMDB/finals/{name}.pt')
data = prunee(data, 2)
v, h = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
object_type(v,h,data, 3)

In [ ]:
for i in range(data.num_relations):
    print(i, data.i2r[i])

In [ ]:

v = torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_ver{node_idx}')
h = torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_hor{node_idx}')
#I want to get the indices of the triples with type relation 
output_indices_v, output_values, value_indices = select_relation(v, data.num_entities, 39)
output_indices_h, output_values, value_indices = select_relation(h, data.num_entities, 39)
objects_types = match_to_triples(output_indices_v, output_indices_h,data, sparse=False)
list = []
for i in objects_types:
    list.append(data.i2e[i[2]][0].split('#')[1])
result = Counter(list)
print(result)
#probably node person is the most frequent - what about I delete all the triples where Person is the object?????


In [ ]:
def select_entity(sparse_tensor,class_id):
    ''' Select the subset of the tensor based on the id of the class to be zeroed out'''
    value_indices = torch.where(sparse_tensor.coalesce().indices() == class_id)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices[1]] = 0
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())

    return masked_sparse_tensor
select_entity(h, 5230)

In [ ]:
5230*39%5230 
#class_id = 5230
#value_indices = torch.where(v.coalesce().indices() == class_id or v.coalesce().indices()%class_id == 0)
v.coalesce().indices()
154915

In [ ]:
def select_one_relation(sparse_tensor,data, relation,value =1):
    """ Selects the values of a sparse tensor based on the relation"""
    sparse_tensor = torch.sparse_coo_tensor(sparse_tensor._indices(), torch.zeros(sparse_tensor._indices().shape[1]), sparse_tensor.size() )
    output_indices, output_values, value_indices=select_relation(sparse_tensor,data.num_entities,relation)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices] = value
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())
    return masked_sparse_tensor

In [ ]:
torch.where(h.coalesce().indices() == torch.tensor(5230))

In [ ]:
def object_type(v,h,data, relation_id = None,type=True):
    ''' Get the object class of a specific relation'''
    if type:
        relation_id = [i for i in range(data.num_relations) if 'type' in data.i2r[i]][-1]
    output_indices_v, output_values, value_indices = select_relation(v, data.num_entities, relation_id)
    output_indices_h, output_values, value_indices = select_relation(h, data.num_entities, relation_id)
    objects_types = match_to_triples(output_indices_v, output_indices_h,data, sparse=False)
    list = []
    for i in objects_types:
        list.append(data.i2e[i[2]][0])#.split('#')[1])
    result = Counter(list)
    return result



for node_idx in data.withheld[:,0]:
    h, v = torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_ver{node_idx}'), torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_hor{node_idx}')
    print(f'node {node_idx}:', object_type(v,h,data, 39))


In [ ]:


relation_id = [i for i in range(data.num_relations) if 'type' in data.i2r[i]]
relation_id




In [ ]:
def select_one_relation(sparse_tensor,data, relation,value =1):
    """ Selects the values of a sparse tensor based on the relation"""
    sparse_tensor = torch.sparse_coo_tensor(sparse_tensor._indices(), torch.zeros(sparse_tensor._indices().shape[1]), sparse_tensor.size() )
    output_indices, output_values, value_indices=select_relation(sparse_tensor,data.num_entities,relation)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices] = value
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())
    return masked_sparse_tensor

In [ ]:

e,n,x = find_n_hop_neighbors(edge_index_oneadj(data.triples) ,2, 5757)
x.shape

In [ ]:
list = []
for i in data.triples:
    if i[0] == 5757:
        list.append(i[2])
    if i[2] == 5757:
        list.append(i[0])

count = []  
for i in data.triples: 
    for j in list:
        if i[0] == j or i[2] == j:
            count.append[]
count

In [ ]:
data.withheld[torch.where(data.withheld[:, 0] == torch.tensor([5757])),1][0]

In [ ]:
classes = d_classes(data)
for c in classes:
    for node in c:
        

In [ ]:
classes = d_classes(data)
count = 0
dict_rel_all_classes = {}
dict_rel = {}
for j in range(len(classes)):
    dict_rel = {}
    for node_idx in classes[j]:
        for i in data.triples:
            if i[0] == node_idx or i[2] == node_idx:
                count += 1
                if data.i2r[int(i[1])] not in dict_rel.keys():
                    dict_rel[data.i2r[int(i[1])]] = 1
                else:
                    dict_rel[data.i2r[int(i[1])]] += 1
    dict_rel_all_classes[j] = dict_rel

count
dict_rel_all_classes

In [ ]:
v_0 = select_on_relation_sparse(v_bin,data, 30)
h_0 = select_on_relation_sparse(h_bin,data, 30)

print(v_0.coalesce().values().count_nonzero(),h_0.coalesce().values().count_nonzero())
out = model.forward2(h_0,v_0)
res = nn.Softmax(dim=0)(out[node_idx])
res

In [ ]:
def select_on_relation_sparse(sparse_tensor,data, relation):
    ''' Selects the values of a sparse tensor based on the relation'''
    output_indices, output_values, value_indices=select_relation(sparse_tensor,data.num_entities,relation)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices] = 0
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())
    return masked_sparse_tensor
  

In [ ]:
def select_one_relation(sparse_tensor,data, relation):
    ''' Selects the values of a sparse tensor based on the relation'''
    sparse_tensor = torch.sparse_coo_tensor(sparse_tensor._indices(), torch.zeros(sparse_tensor._indices().shape[1]), sparse_tensor.size() )
    output_indices, output_values, value_indices=select_relation(sparse_tensor,data.num_entities,relation)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices] = 1
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())
    return masked_sparse_tensor
v, h = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
v = select_one_relation(v,data, 39)
h = select_one_relation(h,data, 39)


In [ ]:
count = 0
types = []
for i in data.triples:
    if i[1]==34:
        types.append(int(i[2]))
        count+=1
count
types = set(types)
types = [data.i2e[i] for i in types]
types

In [ ]:
v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_ver{node_idx}')
h = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_hor{node_idx}')
#loop over keys of counter
for key in Counter(m[:,1].tolist()).keys():
    # v_ = select_on_relation_sparse(v,data, key)
    # h_ = select_on_relation_sparse(h,data, key)
    v_ = select_one_relation(v,data, key)
    h_ = select_one_relation(h,data, key)
    out = model.forward2(h_,v_)
    res = nn.Softmax(dim=0)(out[node_idx])
    #print(f'ypred explain no {data.i2r[key]}, {key}', res)
    if torch.argmax(res)!=torch.argmax(res_full):
        pass
        #print(f'wrong prediction without {data.i2r[key]}')
    else:
        print(f'correct only with {data.i2r[key]}')
        print(f'ypred only with {data.i2r[key]}, {key}', res)



In [ ]:
#define a baseline: no use RGCNExplainer - just rule out relations based on prediction of the model

#node:
node_idx = 5678

#label for that node
if node_idx in data.withheld[:,0]:
    print('ypred true', data.withheld[data.withheld[:,0]==node_idx,1])

#edge index
edge_index = edge_index_oneadj(data.triples)

#number of hops
n_hops = 2

#augment dataset with self loops and inverse relations

hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)

#get the edge index at 2 hops per node
_,_,index_h = find_n_hop_neighbors(hor_graph.coalesce().indices(), n_hops, node_idx)
_,_,index_v = find_n_hop_neighbors(ver_graph.coalesce().indices(), n_hops, node_idx)

h = torch.sparse_coo_tensor(index_h, torch.ones(index_h.shape[1]), hor_graph.size() )
v = torch.sparse_coo_tensor(index_v, torch.ones(index_v.shape[1]), ver_graph.size() )

#match to triple
m = match_to_triples(v,h, data)

#counter of relations in the 2 hops subgraph
Counter(m[:,1].tolist())

#forward pass of the model
out = model.forward2(h,v)
res = nn.Softmax(dim=0)(out[node_idx])
print(f'ypred explain all subgraph: {torch.argmax(res)} with prediction probability: {res}')


In [ ]:
Counter(m[:,1].tolist())

v = select_one_relation(v,data, 39)
h = select_one_relation(h,data, 9)
h.coalesce().values().count_nonzero()

In [ ]:
node_idx = 5757
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[node_idx, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)

m = match_to_triples(ver_graph,hor_graph,data, node_idx)
v, h = ver_graph, hor_graph
for key in Counter(m[:,1].tolist()).keys():
    v_ = select_on_relation_sparse(v,data, key)
    h_ = select_on_relation_sparse(h,data, key)
    out = model.forward2(h_,v_)
    res = nn.Softmax(dim=0)(out[node_idx])
    #print(f'ypred explain no {data.i2r[key]}, {key}', res)
    if torch.argmax(res)!=torch.argmax(res_full):
        print(f'for node {node_idx}, wrong prediction without {data.i2r[key]}')

In [ ]:
name = 'IMDb_us'
data = torch.load(f'data/IMDB/finals/{name}.pt')

data = prunee(data, 2)
data.triples = torch.Tensor(data.triples).to(int)
data.withheld = torch.Tensor(data.withheld).to(int)
data.training = torch.Tensor(data.training).to(int)

In [ ]:
numbers_to_remove = []
for i in range(data.num_relations):
    if 'genre' in data.i2r[i]:
        print(f'{i}: {data.i2r[i]}')
        numbers_to_remove.append(i)
numbers_to_remove

In [ ]:
# for i in data.triples:
#     for j in numbers_to_remove:
#         if i[1] == j:
#             print(i)
for i in data.triples:
    if i[0] ==9662:
        print(data.i2e[i[0]], data.i2r[i[1]], data.i2e[i[2]])

In [ ]:
name = 'IMDb_most_genre'
data = torch.load(f'data/IMDB/finals/{name}.pt')

data = prunee(data, 2)
data.triples = torch.Tensor(data.triples).to(int)
data.withheld = torch.Tensor(data.withheld).to(int)
data.training = torch.Tensor(data.training).to(int)

In [ ]:
data.withheld

In [ ]:
data.i2e[3599]

In [ ]:
data.withheld[data.withheld[:,0]==10241,1]
data.withheld[torch.where(data.withheld[:, 0] == torch.tensor([10112])),1]

In [ ]:
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
get_relations(data)
relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]
    
relations = ['label', 'node_idx','number_neighbors', 'prediction_explain', 'prediction_full', 'prediction_explain_binary'] + relations
df = pd.DataFrame(columns=relations)
d = d_classes(data)

#count how many nodes per class
count = {}
for i in range(len(d)):
    count[i] = len(d[i])
count



In [ ]:
for key in Counter(m[:,1].tolist()).keys():
    v_ = select_on_relation_sparse(v,data, key)
    h_ = select_on_relation_sparse(h,data, key)
    out = model.forward2(h_,v_)
    res = nn.Softmax(dim=0)(out[node_idx])
    print(f'ypred explain no {data.i2r[key]}, {key}', res)
    if torch.argmax(res)!=torch.argmax(res_full):
        print(f'wrong prediction without {data.i2r[key]}')

In [ ]:
#get the node of the most important relations - the relations with the highest weights

tensor_list = (list(v.coalesce().indices()[1][v.coalesce().values()>0.5]) + list(h.coalesce().indices()[0][h.coalesce().values()>0.5]))
float_list = [tensor.item() for tensor in tensor_list]
len(set(float_list))


In [ ]:
count = 0
others = []
for i in data.triples:

    if i[0] == 5857:
        print(i)
        count += 1
        others.append(i[2])
    if i[2] == 5857:
        print(i)
        count += 1
        others.append(i[0])

count
others

In [ ]:
count = 0
l = []
a = []
for i in data.triples:
    for n in others:
        if i[0] == int(n) or i[2] == int(n):
            count += 1
            l.append(n)
            a.append(i)
            #print(i)
            break
res = set(l)
resa = set(a)
print(len(resa))            
print(len(res))
count
#others

In [ ]:
def find_repeating_sublists(sublists):
    repeating_elements = {}

    for sublist in sublists:
        key1 = (sublist[0], sublist[2])
        key2 = (sublist[2], sublist[0])

        if key1 in repeating_elements:
            repeating_elements[key1].append(sublist[1])
        elif key2 in repeating_elements:
            repeating_elements[key2].append(sublist[1])
        else:
            repeating_elements[key1] = [sublist[1]]

    result_array = []
    for key, values in repeating_elements.items():
        if len(values) > 1:
            result_array.append([key[0], values, key[1]])
        else:
            result_array.append([key[0], [values[0]], key[1]])

    return result_array

def unnest_list(nested_list):
    return [item for sublist in nested_list for item in (unnest_list(sublist) if isinstance(sublist, list) else [sublist])]

def visualize(node_idx, n_hop, data, masked_ver,threshold,name, result_weights=True, low_threshold=False ):
    """ 
    Visualize important nodes for node idx prediction
    """
    dict_index = dict_index_classes(data,masked_ver)
    
    #select only nodes with a certain threshold
    sel_masked_ver = sub_sparse_tensor(masked_ver, threshold,data, low_threshold)
    if len(sel_masked_ver)==0:
        sel_masked_ver=sub_sparse_tensor(masked_ver, 0,data, low_threshold)
    print('sel masked ver',sel_masked_ver)
    indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) #original edge indexes

    
    
    G = nx.Graph()
    if result_weights:
        values = sel_masked_ver.coalesce().values().detach().numpy()
        for s,p,o in zip(indices_nodes[0],values , indices_nodes[1]):
            G.add_edge(int(s), int(o), weight=np.round(p, 2))

    else:

        triples_matched = match_to_triples(sel_masked_ver, data)
        l = []
        for i in triples_matched[:,1]:
            l.append(data.i2rel[int(i)][0])
        triples_matched = find_repeating_sublists(triples_matched.numpy())
        for s,p,o in triples_matched:
            G.add_edge(int(s), int(o), weight=p)


    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    weights = [[item] if not isinstance(item, list) else item for item in weights]


    pos = nx.circular_layout(G)

    ordered_dict = {}
    for item in list(G.nodes):
        if item in ordered_dict:
            ordered_dict[item].append(dict_index[item])
        else:
            ordered_dict[item] =  dict_index[item]

    dict_index = ordered_dict

    labeldict = {}
    for node in G.nodes:
        labeldict[int(node)] = int(node)  


    dict = {}
    for k,v in dict_index.items():
        for k1,v1 in data.entities_classes.items():
            if v==k1: 

                dict[k] = v1
            else:
                if k not in dict:
                    dict[k] = 0
                

    color_list = list(dict.values())
    color_list = list(encode_dict(dict_index).values())


    col_weights = [weights[i][0] for i in range(len(weights))]
    if result_weights:
        
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=col_weights, node_color =  color_list, cmap="Set2",edge_cmap=plt.cm.Reds,font_size=8)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=nx.get_edge_attributes(G,'weight'),font_size=8,font_color='red')
        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
        sm.set_array(weights)
        cbar = plt.colorbar(sm)
        cbar.ax.set_title('Weight')
        plt.title("Node {}'s {}-hop neighborhood important nodes".format(node_idx, n_hop))
    else:
        rel = nx.get_edge_attributes(G,'weight')
        rel = {k: [data.i2rel[i][0] for i in v] for k,v in rel.items()}
        col_weights = [sum(weights[i], 3) if len(weights[i]) > 1 else weights[i][0] for i in range(len(weights))]
        nx.draw(G, pos,labels = labeldict, edge_color=col_weights,edgelist=edges,node_color =  color_list, cmap="Set2",font_size=7, arrows = True)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=rel,font_size=8,font_color='red')
        
        res = Counter(unnest_list(rel.values()))
    if result_weights:
        if not os.path.exists(f'chk/{name}_chk/graphs'):
            os.makedirs(f'chk/{name}_chk/graphs')  
        plt.savefig(f'chk/{name}_chk/graphs/Explanation_{node_idx}_{n_hop}_weights.png')

        #plt.show()

    else:
        if not os.path.exists(f'chk/{name}_chk/graphs'):
            os.makedirs(f'chk/{name}_chk/graphs')  
        plt.savefig(f'chk/{name}_chk/graphs/Explanation_{node_idx}_{n_hop}_relations.png')    
        #plt.show()
        return res, weights
    

In [ ]:

v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/masked_adj/masked_ver{node_idx}_new')
res, weights = visualize(node_idx, 2, data, v, 0, name, result_weights=False, low_threshold=False)

In [ ]:
import torch

def hor_ver_graph(triples, n, r):
    """
    input: triples, number of nodes, number of relations
    output: hor_graph, ver_graph : horizontally and vertically stacked adjacency matrix
    """

    hor_ind, hor_size = adj(triples, n, 2*r+1, vertical=False)
    ver_ind, ver_size = adj(triples, n, 2*r+1, vertical=True)

    rn, _ = hor_size  # horizontally stacked adjacency matrix size
    r = rn // n  # number of relations enriched divided by number of nodes

    vals = torch.ones(ver_ind.size(0), dtype=torch.float)  # number of enriched triples

    hor_graph = torch.sparse.FloatTensor(hor_ind.t(), vals, hor_size)  # size: n, r, emb
    ver_graph = torch.sparse.FloatTensor(ver_ind.t(), vals, ver_size)

    return hor_graph, ver_graph

def adj(triples, num_nodes, num_rels, cuda=False, vertical=True):
    """
     Computes a sparse adjacency matrix for the given graph (the adjacency matrices of all
     relations are stacked vertically).

     :param edges: List representing the triples
     :param i2r: list of relations
     :param i2n: list of nodes
     :return: sparse tensor
    """
    r, n = num_rels, num_nodes
    size = (r * n, n) if vertical else (n, r * n)

    from_indices = []
    upto_indices = []

    for s, p, o in triples:
        offset = p.item() * n
        s = offset + s.item() if vertical else s.item()
        o = offset + o.item() if not vertical else o.item()
        from_indices.append(s)
        upto_indices.append(o)

    indices = torch.tensor([from_indices, upto_indices], dtype=torch.long, device=d(cuda))

    return indices.t(), size



In [ ]:
adj(data.triples, data.num_entities, data.num_relations, cuda=False, vertical=True)
hor_ver_graph(data.triples, data.num_entities, data.num_relations)

In [ ]:

hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
edge_h, edge_v = hor_graph.coalesce().indices(), ver_graph.coalesce().indices()
_,_,sub_edges_tensor_h  = find_n_hop_neighbors(edge_h,2, 5699)
_,_,sub_edges_tensor_v  = find_n_hop_neighbors(edge_v,2, 5699)


In [ ]:
len(sub_edges_tensor_v)

In [ ]:
sub_edges_tensor
indexes = sub_edges_tensor%data.num_entities
indexes
r = sub_edges_tensor//data.num_entities
r

In [ ]:
_,p = torch.div(sub_edges_tensor, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
s,o = sub_edges_tensor%data.num_entities
result = torch.stack([s,p,o], dim=1)
result

In [ ]:
#try initialization where the first hop neighborhood gets initialized with 1s and the rest with different methods??
def construct_edge_mask(self, num_nodes,sparse_tensor,data, const_val=1.0, relation_id = 2):
    """
    Construct edge mask
    """
    init_strategy = self.init_strategy
    # if num_nodes > 1000:
    #     init(strategy="const", const_val=0.1)
    data = self.data
    num_entities = data.num_entities
    torch.manual_seed(42)
    mask = nn.Parameter(torch.FloatTensor(num_nodes))

    if init_strategy == "normal":
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
    elif init_strategy == "const":
        nn.init.constant_(mask, const_val) 
    elif init_strategy == "zero_out":
        '''initialize the mask with the zero out strategy: we zero out edges belonging to specific relations'''
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
        output_indices, output_values, value_indices=select_relation(sparse_tensor,relation_id)
        _,_,value_indices1=select_relation(sparse_tensor,33)
        print(value_indices, value_indices1)
        value_indices = torch.cat((value_indices, value_indices1), 0)
        mask.data[[value_indices]] = 0
    

    elif init_strategy == "overall_frequency":
        '''Initialize the mask with the overall frequency of the relations'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        overall_rel_frequency = dict(Counter(data.triples[:,1].tolist()))#.most_common()

        overall_rel_frequency_  = {key: round(value/len(data.triples[:,1].tolist()),5) for key, value in overall_rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = overall_rel_frequency_[i]
    
    elif init_strategy == "relative_frequency":
        ''' Initialize the mask with the relative frequency of the relations-relative for the node to be explained'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        rel_frequency = dict(Counter(p))
        rel_frequency_  = {key: round(value/len(p),5) for key, value in rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = rel_frequency_[i]

    elif init_strategy == "inverse_relative_frequency":
        ''' Initialize the mask with the relative frequency of the relations-relative for the node to be explained'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        rel_frequency = dict(Counter(p))
        rel_frequency_  = {key: 1 - round(value/len(p),5) for key, value in rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = rel_frequency_[i]


    elif init_strategy == "domain_frequency":
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
        for i in p:

            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = dict_domain[i]

    elif init_strategy == "range_frequency":
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
        for i in p:
                _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
                mask.data[[value_indices]] = dict_range[i]
    elif init_strategy == "rdf":
        rdf = [i for i in range(data.num_relations) if 'rdf' in data.i2r[i]]
        for i in rdf:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = 0
    elif init_strategy == "owl":
        owl = [i for i in range(data.num_relations) if 'owl' in data.i2r[i]]
        for i in owl:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = 0
    print(f'mask initialized with {init_strategy} strategy: {mask}')   
    return mask

In [ ]:
import torch
a = torch.randn(2, 3).to_sparse().requires_grad_(True)
print(a)
#a.values = torch.ones_like(a.values())
values = torch.ones(a._nnz())
#len(a.values)
a.indices()
sparse_tensor = torch.sparse_coo_tensor(a.indices(), torch.ones(a._nnz()), a.size(), requires_grad=True)
sparse_tensor

In [ ]:

v  = torch.load('chk/aifb_chk/size_0.05_lr_0.1_epochs_30_threshold_0.7_init_normal/masked_adj/masked_ver5757')
h = torch.load('chk/aifb_chk/size_0.05_lr_0.1_epochs_30_threshold_0.7_init_normal/masked_adj/masked_hor5757')

In [ ]:
v

In [ ]:
nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() > 0.7]
nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
nonzero_values = v.coalesce().values()[v.coalesce().values() > 0.7]
sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
sel_masked_ver

In [ ]:
v = sub_sparse_tensor(v, 0.5, data, low_threshold=False)
h = sub_sparse_tensor(h, 0.5, data, low_threshold=False)

In [ ]:
def sub(v):
    nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() < 0.5]
    nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
    nonzero_values = v.coalesce().values()[v.coalesce().values() < 0.5]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver
    

In [ ]:
def sub(v, threshold):
    nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() > threshold]
    nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
    nonzero_values = v.coalesce().values()[v.coalesce().values() > threshold]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver
sub(v, 0.5)

In [ ]:
def match_to_triples(v,h, data, sparse=True):
    if sparse:
        pv,_ = torch.div(v.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sv,ov = v.coalesce().indices()%data.num_entities
        result_v = torch.stack([sv,pv,ov], dim=1)
        ph,_ = torch.div(h.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sh,oh = h.coalesce().indices()%data.num_entities
        result_h = torch.stack([sh,ph,oh], dim=1)
        result = torch.cat((result_v, result_h), 0)


                    
    else:
        if len(h )!= 0:
            _,ph = torch.div(h, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sh,oh = h%data.num_entities
            result_h = torch.stack([sh,ph,oh], dim=1)
        if len(v)!=0:
            pv, _ = torch.div(v, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sv,ov = v%data.num_entities
            result_v = torch.stack([sv,pv,ov], dim=1)
        if len(h) != 0 and len(v) != 0:
            result = torch.cat((result_v, result_h), 0)
            print(pv,ph)
        if len(h) == 0:
            result = result_v
            print(pv)
        if len(v) == 0:
            result = result_h
            print(ph)
        

                    
    
    return result

m = match_to_triples(v,h, data, sparse=True)
Counter(m[:,1].tolist())




In [ ]:
def match_to_triples(v, data, sparse=True):
    if sparse:
        # p,_ = torch.div(v.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        # s,o = v.coalesce().indices()%data.num_entities
        # result = torch.stack([s,p,o], dim=1)
        matching = []
        indexes = v.coalesce().indices()%data.num_entities
        for j in range(indexes.size()[1]):
            for triple in data.triples:
                if triple[0] == indexes[0][j] and triple[2] == indexes[1][j]:
                    matching.append(triple)
        result = torch.stack(matching)

                    
    else:
        matching = []
        for i,i2 in zip(v[:,0],v[:,1]):
            for j,j1,j2, index in zip(data[:,0],data[:,1],  data[:,2], range(len(data[:,0]))):
                if i == j and i2 == j2:
                    matching.append(data[index])
                    

        result = torch.stack(matching)
    
    return result
m = match_to_triples(h, data, sparse=True)
Counter(m[:,1].tolist())

In [ ]:
def sub(v, threshold):
    nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() > threshold]
    nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
    nonzero_values = v.coalesce().values()[v.coalesce().values() > threshold]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver

def visualize(node_idx, n_hop, data, masked_ver,masked_hor, threshold,name, result_weights=True, low_threshold=False,experiment_name=None ):
    """ 
    Visualize important nodes for node idx prediction
    """
    dict_index = dict_index_classes(data,masked_ver)
    mask = torch.vstack((masked_ver, masked_hor.t()))
    mask = sub(mask, threshold)
    print(mask)
    #select only nodes with a certain threshold
    sel_masked_ver = sub(masked_ver, threshold)
    sel_masked_hor = sub(masked_hor, threshold)
    if len(sel_masked_ver)==0:
        sel_masked_ver=sub_sparse_tensor(masked_ver, 0,data, low_threshold)
    #mask = torch.vstack((sel_masked_ver, sel_masked_hor.t()))
    print('sel masked ver',mask)
    indices_nodes = mask.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) #original edge indexes

    
    
    G = nx.Graph()
    if result_weights:
        values = mask.coalesce().values().detach().numpy()
        for s,p,o in zip(indices_nodes[0],values , indices_nodes[1]):
            G.add_edge(int(s), int(o), weight=np.round(p, 2))

    else:

        triples_matched = match_to_triples(sel_masked_ver,sel_masked_hor, data)
        l = []
        for i in triples_matched[:,1]:
            l.append(data.i2rel[int(i)][0])
        triples_matched = find_repeating_sublists(triples_matched.numpy())
        print(triples_matched)
        for s,p,o in triples_matched:
            G.add_edge(int(s), int(o), weight=p)


    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    weights = [[item] if not isinstance(item, list) else item for item in weights]


    pos = nx.circular_layout(G)

    ordered_dict = {}
    for item in list(G.nodes):
        if item in ordered_dict:
            ordered_dict[item].append(dict_index[item])
        # else:
        #     ordered_dict[item] =  dict_index[item]

    dict_index = ordered_dict

    labeldict = {}
    for node in G.nodes:
        labeldict[int(node)] = int(node)  


    dict = {}
    for k,v in dict_index.items():
        for k1,v1 in data.entities_classes.items():
            if v==k1: 

                dict[k] = v1
            else:
                if k not in dict:
                    dict[k] = 0
                

    color_list = list(dict.values())
    color_list = list(encode_dict(dict_index).values())


    col_weights = [weights[i][0] for i in range(len(weights))]
    if result_weights:
        
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=col_weights, node_color =  color_list, cmap="Set2",edge_cmap=plt.cm.Reds,font_size=8)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=nx.get_edge_attributes(G,'weight'),font_size=8,font_color='red')
        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
        sm.set_array(weights)
        cbar = plt.colorbar(sm)
        cbar.ax.set_title('Weight')
        plt.title("Node {}'s {}-hop neighborhood important nodes".format(node_idx, n_hop))
    else:
        rel = nx.get_edge_attributes(G,'weight')
        rel = {k: [data.i2rel[i][0] for i in v] for k,v in rel.items()}
        col_weights = [sum(weights[i], 3) if len(weights[i]) > 1 else weights[i][0] for i in range(len(weights))]
        nx.draw(G, pos,labels = labeldict, edge_color=col_weights,edgelist=edges,node_color =  color_list, cmap="Set2",font_size=7, arrows = True)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=rel,font_size=8,font_color='red')
        
        res = Counter(unnest_list(rel.values()))
        print(res)
    if result_weights:
        if not os.path.exists(f'chk/{name}_chk/{experiment_name}⁄graphs'):
            os.makedirs(f'chk/{name}_chk/{experiment_name}⁄graphs')  
        plt.savefig(f'chk/{name}_chk/{experiment_name}⁄graphs/Explanation_{node_idx}_weights.png')

        #plt.show()

    else:
        if not os.path.exists(f'chk/{name}_chk/{experiment_name}⁄graphs'):
            os.makedirs(f'chk/{name}_chk/{experiment_name}⁄graphs')  
        plt.savefig(f'chk/{name}_chk/{experiment_name}⁄graphs/Explanation_{node_idx}_relations.png')    
        #plt.show()
        return res, weights
    


In [ ]:
torch.vstack((v,h.t()))

In [ ]:
visualize(node_idx, 2, data, v,h, 0.5,name, result_weights=False, low_threshold=False,experiment_name=None )

In [ ]:
def match_to_triples(v,h, data, sparse=True):
    if sparse:
        pv,_ = torch.div(v.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sv,ov = v.coalesce().indices()%data.num_entities
        result_v = torch.stack([sv,pv,ov], dim=1)
        ph,_ = torch.div(h.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sh,oh = h.coalesce().indices()%data.num_entities
        result_h = torch.stack([sh,ph,oh], dim=1)
        result = torch.cat((result_v, result_h), 0)


                    
    else:

        _,ph = torch.div(h, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sh,oh = h%data.num_entities
        result_h = torch.stack([sh,ph,oh], dim=1)

        pv, _ = torch.div(v, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sv,ov = v%data.num_entities
        result_v = torch.stack([sv,pv,ov], dim=1)

        result = torch.cat((result_v, result_h), 0)

        if len(h )!= 0:
            _,ph = torch.div(h, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sh,oh = h%data.num_entities
            result_h = torch.stack([sh,ph,oh], dim=1)
        if len(v)!=0:
            pv, _ = torch.div(v, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sv,ov = v%data.num_entities
            result_v = torch.stack([sv,pv,ov], dim=1)
        if len(h) != 0 and len(v) != 0:
            result = torch.cat((result_v, result_h), 0)
            print(pv,ph)
        if len(h) == 0:
            result = result_v
            print(pv)
        if len(v) == 0:
            result = result_h
            print(ph)
        

                    
    
    return result

In [ ]:
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
n_hops = 0
node_idx = 5678
#hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
edge_index_h, edge_index_v = hor_graph.coalesce().indices(), ver_graph.coalesce().indices()
sub_edges, neighbors, sub_edges_tensor_h  = find_n_hop_neighbors(edge_index_h, n=n_hops, node=node_idx)

sub_edges, neighbors, sub_edges_tensor_v  = find_n_hop_neighbors(edge_index_v, n=n_hops, node=node_idx)
print(len(list(neighbors)))
print('shape sub',sub_edges_tensor_h.shape, sub_edges_tensor_v.shape)

In [ ]:
node_idx = 5677
count = 0



for m in data.triples:
    if m[0] == node_idx or m[2] == node_idx:
        print(m)
        count += 1
print(count)

In [ ]:
#initialize the mask to give priority to the 1st hop relations
neighbors_h, neighbors_v = list((1,1)), list((2,3))
neighbors = len(set(neighbors_v + neighbors_h))
len(neighbors)
#set(list((neighbors_h, neighbors_v)))


In [ ]:
#separate the graph into subgraphs per relation 
#create dictionary where each key is a relation and each value is a list of the triples with that relation
#each value is a tensor with the edge indices of the triples with that relation

dict_rel = {}
for i in range(data.num_relations):
    dict_rel[i] = []
for i in range(len(data.triples)):

    dict_rel[int(data.triples[i][1])].append(torch.tensor([data.triples[i][0], data.triples[i][2]]))

dict_rel